# Deploying Time-Series Statistical Model on BEntoML

The first step of creating a prediction service with BentoML, is to write a prediction service class inheriting from bentoml.BentoService, and specify the required model artifacts, PyPI dependencies and write the service API function. Here is a minimal prediction service definition with BentoML:



In [4]:
%%writefile holt.py

# holt.py
from bentoml import env, artifacts, api, BentoService
from bentoml.handlers import DataframeHandler,DataframeInput
from bentoml.artifact import PickleArtifact
import numpy as np

@env(pip_dependencies=["statsmodels==0.10.1","joblib","numpy"],conda_dependencies=["ruamel.yaml==0.16"])

@artifacts([PickleArtifact('model')])
class holt(BentoService):
    @api(input=DataframeInput())
    def predict(self, df):

        # Printing the dataframe to cross-cjheck
        print(df.head())

        # Parsing the dataframe values
        weeks=int(df.iat[0,0])
        print(weeks)
        return((self.artifacts.model).forecast(weeks))
  

Writing holt.py


The bentoml.api decorator defines a service API, which is the entry point for accessing the prediction service. The DataframeInput here denotes that this service API will convert HTTP JSON request into pandas.DataFrame object before passing it to the user-defined API function code for inference.

The pip_dependencies specify the libraries that you would need in your code. Any library specified here will automatically get added to the requirements.txt folder . 

Here we're using the PickleArtifact. However,  BentoML also provide model artifact for other frameworks such as PytorchModelArtifact, KerasModelArtifact, FastaiModelArtifact, and XgboostModelArtifact etc.

The following code trains a scikit-learn model and bundles the trained model with an Holt instance. The Holt instance is then saved to disk in the BentoML SavedBundle format, which is a versioned file archive that is ready for production models serving deployment,we've considered a shampoo sales data which is available publicly.

In [5]:
%%writefile main.py


#main.py

import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import numpy as np
import joblib
from holt import holt

df=pd.read_csv('https://raw.githubusercontent.com/jbrownlee/Datasets/master/shampoo.csv')

#Taking a test-train split of 80 %
train=df[0:int(len(df)*0.8)] 
test=df[int(len(df)*0.8):]

#Pre-processing the  Month  field
train.Timestamp = pd.to_datetime(train.Month,format='%m-%d') 
train.index = train.Timestamp 
test.Timestamp = pd.to_datetime(test.Month,format='%m-%d') 
test.index = test.Timestamp 

#fitting the model based on  optimal parameters
model = ExponentialSmoothing(np.asarray(train['Sales']) ,seasonal_periods=7 ,trend='add', seasonal='add',).fit()

#creating an instance of the holt class
holt_obj = holt()


# Pack the newly trained model artifact
holt_obj.pack('model', model)
saved_path = holt_obj.save()

Writing main.py


You can start a REST API server by specifying the BentoService’s name and version, or provide the file path to the saved bundle:



In [ ]:
!bentoml serve holt:latest

The IrisClassifier model is now served at localhost:5000,replace 2 with the number of weeks fow which you want a prediction  . Use curl command to send a prediction request:

curl -i \
  --header "Content-Type: application/json" \
  --request POST \
  --data '[[2]]' \
  http://localhost:5000/predict

BentoML provides a convenient way to containerize the model API server with Docker:

1.Find the SavedBundle directory with bentoml get command

2.Run docker build with the SavedBundle directory which contains a generated Dockerfile

3.Run the generated docker image to start a docker container serving the model



In [ ]:
!saved_path=$(bentoml get IrisClassifier:latest -q | jq -r ".uri.uri")

In [ ]:
!docker build -t {docker_username}/iris-classifier $saved_path

In [ ]:
!docker run -p 5000:5000 -e BENTOML_ENABLE_MICROBATCH=True {docker_username}/iris-classifier

You can check out more on deploying the model on cluster at:<br>
1.https://docs.bentoml.org/en/stable/quickstart.html#deploy-api-server-to-the-cloud<br>
2.https://www.kubeflow.org/docs/components/serving/bentoml/ (if you have kubeflow installed)